In [1]:
%pip install langchain_huggingface
%pip install pinecone pinecone-client
%pip install -U langchain-teddynote

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. embedding

In [32]:
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()

True

In [33]:
from langchain.text_splitter import CharacterTextSplitter

def get_text_chunks(text, chunk_size = 1000, chunk_overlap=200):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [34]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings


def embedding_doc(chunks, model_name):

    hf_embeddings = HuggingFaceEndpointEmbeddings(
        model=model_name,
        task="feature-extraction",
        huggingfacehub_api_token=os.getenv("HF_API_TOKEN"),
    )

    LIMIT = 30
    if len(chunks)>30:
       embedded_documents = []
       for i in range(0, len(chunks)+1, 30):
            embedded_doc_part = hf_embeddings.embed_documents(chunks[i:min(i+30, len(chunks))])
            embedded_documents += embedded_doc_part
    else:
        embedded_documents = hf_embeddings.embed_documents(chunks)

    return embedded_documents, hf_embeddings

# 2. Store in Pinecone

In [35]:
from pinecone import *
load_dotenv()  # Load environment variables from .env file

class PineconeStore:
    def __init__(self):
        # Load API key from environment variables
        pinecone_api_key = os.getenv("PINECONE_API_KEY")
        if not pinecone_api_key:
            raise ValueError("PINECONE_API_KEY environment variable not set")

        # Create a Pinecone instance
        self.pc = Pinecone(api_key=pinecone_api_key)

        # Define the index name
        self.index_name = "iskku-data"

        # Check if the index exists, if not create it
        if self.index_name not in self.pc.list_indexes().names():
            self.pc.create_index(
                name=self.index_name,
                dimension=1024,
                metric='cosine',
                spec=ServerlessSpec(cloud='aws', region='us-east-1')
            )

    def save_vectors(self, vectors, file_path, chunks):
        # Get the index
        index = self.pc.Index(self.index_name)

        # Iterate over the embeddings and save each one with unique metadata
        for i, vector in enumerate(vectors):
            vector_id = f'{os.path.basename(file_path)}chunk_{i}'  # Unique ID for each chunk
            chunk_metadata = {
                "id": vector_id,
                "source": os.path.basename(file_path),
                "chunk": i,
                "text": chunks[i]  # Add the text of the chunk here
            }
            # Upsert each vector with its corresponding metadata
            index.upsert(
                vectors=[(vector_id, vector, chunk_metadata)],
                namespace=os.path.basename(file_path))

In [36]:
import glob

file_path = './results/*.txt'
txt_files = glob.glob(file_path)

In [37]:
import string
import re    

def file_to_string(file_path):

    json_list_pattern = r'\[.*?\]'
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        tabular_data = re.findall(json_list_pattern, content, re.DOTALL)
        text_data = re.sub(json_list_pattern, '', content, flags=re.DOTALL)
    return text_data, tabular_data

def extract_ascii_printable(s):
    return ''.join(c for c in s if c in string.printable)

In [39]:
for f in txt_files:
    
    print(f'{f} embedding start!')
    f_text, f_tabular = file_to_string(f)
    chunks = get_text_chunks(f_text)
    chunks += f_tabular
    embeddings, hf_embeddings = embedding_doc(chunks, model_name = "intfloat/multilingual-e5-large-instruct")
    
    # avoid ascii unprintable char
    rename_f = extract_ascii_printable(f)
    print(f'storing {rename_f} in pinecone...')
    vector_store = PineconeStore()
    
    vector_store.save_vectors(embeddings, rename_f, chunks)
    print('Success storing in pinecone!')

print('all files are uploaded')

./results\Q & A    FAQ 게시판읽기 ( A partial tuition fee refund for the withdrawn course .  )   성균관대학교 외국인유학생지원팀.txt embedding start!
storing ./results\Q & A    FAQ  ( A partial tuition fee refund for the withdrawn course .  )    .txt in pinecone...
Success storing in pinecone!
./results\Q & A    FAQ 게시판읽기 ( Grade withdrawal )   성균관대학교 외국인유학생지원팀.txt embedding start!
storing ./results\Q & A    FAQ  ( Grade withdrawal )    .txt in pinecone...
Success storing in pinecone!
./results\Q & A    FAQ 게시판읽기 ( Retaking a course )   성균관대학교 외국인유학생지원팀.txt embedding start!
storing ./results\Q & A    FAQ  ( Retaking a course )    .txt in pinecone...
Success storing in pinecone!
./results\성균관대학교 외국인유학생지원팀  Academics  Academic Policies for Intl ' s Students.txt embedding start!
storing ./results\   Academics  Academic Policies for Intl ' s Students.txt in pinecone...
Success storing in pinecone!
./results\성균관대학교 외국인유학생지원팀  Academics  Combined Undergraduate  &  Graduate program.txt embedding start!
storing .

Created a chunk of size 1394, which is longer than the specified 1000


Success storing in pinecone!
./results\성균관대학교 외국인유학생지원팀  Immigration Guide  Insurance.txt embedding start!
storing ./results\   Immigration Guide  Insurance.txt in pinecone...
Success storing in pinecone!
./results\성균관대학교 외국인유학생지원팀  Immigration Guide  Part - time works.txt embedding start!
storing ./results\   Immigration Guide  Part - time works.txt in pinecone...
Success storing in pinecone!
./results\성균관대학교 외국인유학생지원팀  Immigration Guide  Visa.txt embedding start!
storing ./results\   Immigration Guide  Visa.txt in pinecone...
Success storing in pinecone!
all files are uploaded
